In [19]:
#default_exp ghtop

# ghtop

> I cannot believe this library name is not already taken on pypi.

In [20]:
#export
import time, datetime, pytz, sys, signal, shutil, os, json, urllib.request
import enlighten, emoji, blessed, requests, webbrowser
from ghtop.dashing import *
from urllib.parse import parse_qs

from collections import defaultdict

from fastcore.utils import *
from ghapi.all import *

In [21]:
term = blessed.Terminal()
logfile = Path("log.txt")
url = "https://api.github.com/events"

In [22]:
def github_auth_device():
    auth = GhDeviceAuth()
    wb = input(f"""First copy your one-time code: {term.yellow}{auth.user_code}{term.normal}
    Then visit {auth.verification_uri} in your browser, and paste the code when prompted.
    Shall we try to open the link for you? [y/n] """)
    if wb[0].lower()=='y': auth.open_browser()

    print ("Waiting for authorization...", end='')
    token = auth.wait(lambda: print('.', end=''))
    print("Authenticated with GitHub!")
    return token

In [23]:
#export
def exit(msg):
    print(msg, file=sys.stderr)
    sys.exit()

In [24]:
#export
def get_token():
    path = Path.home()/".ghtop_token"
    if path.is_file():
        try: return path.read_text().strip()
        except: exit("Error reading token")

    token = github_auth_device()
    path.write_text(token)
    return token

In [25]:
token = get_token()

In [26]:
token

'84dedf981b71f6d9d50a98d9a67421a61c469ef4'

In [27]:
def limit_cb(rem,quota):
    if remaining_apis >= 1000: return
    print("WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING ")
    print("Remaining calls: " + str(remaining_apis))
    print("WARNING WARNING WARNING WARNING WARNING WARNING WARNING WARNING ")

In [28]:
api = GhApi(limit_cb=limit_cb)

In [29]:
#export
def fetch_events(): return api.activity.list_public_events(per_page=100)

In [30]:
#export
def read_json_log():
    try: return logfile.read_json()
    except FileNotFoundError: return []

In [31]:
printed_event_ids = {}

In [32]:
def show(s): print(emoji.emojize(s, use_aliases=True))

In [78]:
ev = [o for o in fetch_events() if o.type=='PullRequestEvent']

In [79]:
[(i,o.type) for i,o in enumerate(ev)]

[(0, 'PullRequestEvent'),
 (1, 'PullRequestEvent'),
 (2, 'PullRequestEvent'),
 (3, 'PullRequestEvent'),
 (4, 'PullRequestEvent'),
 (5, 'PullRequestEvent'),
 (6, 'PullRequestEvent'),
 (7, 'PullRequestEvent'),
 (8, 'PullRequestEvent'),
 (9, 'PullRequestEvent')]

In [81]:
printed_event_ids={}
for e in ev: print_event(e,None)

✨ asubb opened a pull request on repo WaveBeans/wavebeans ("Sample vector improvements in API...")
✨ boemekeld opened a pull request on repo bohr-cloud/testepr ("Update aaaa...")
✨ talevy opened a pull request on repo elastic/elasticsearc ("move rollup_index param out of RollupActionConfig...")
✅ jurek91 closed a pull request on repo RobertK3000/giftapes ("small tweaks...")
✨ DGryanko opened a pull request on repo DGryanko/Resume_v2 ("Calculator my...")
✨ gustavocaraciolo opened a pull request on repo gustavocaraciolo/pat ("Add entities using the JDL model `patrimonio`...")
✨ xiangjingli opened a pull request on repo operator-framework/c ("Upgrade Multicluster subscription operator to V0.2...")
✨ leonidkit opened a pull request on repo leonidkit/thinknetic ("Добавлено десятое дз...")
✨ hiroto940 opened a pull request on repo inovange/inovange-si ("member link...")


In [69]:
def _title(d): return f'("{d.title[:50]}...")'
def _issue(d): return f'issue # {d.number}'
def _repo(r,i=22): return f'on repo {r[:i]}'

In [76]:
#export
def print_event(e, commits_counter):
    if e.id in printed_event_ids: return
    printed_event_ids[e.id] = 1
    login,repo = e.actor.login,e.repo.name
    # Don't print bot activity (there is a lot!)
    if "bot" in login or "b0t" in login: return

    if e.type == "ReleaseEvent": show(term.firebrick3(f':rocket: {login} released {e.payload.release.tag_name} of {repo}'))
    elif e.type in ("PublicEvent","ForkEvent","CreateEvent","MemberEvent"): return
    elif e.type == "IssuesEvent":
        action,issue = e.payload.action,e.payload.issue
        if action == 'closed':
            show(f':star: {login} closed issue # {issue.number} {_repo(r)} {_title(issue)}')
        elif action == 'opened':
            show(f':closed_mailbox_with_raised_flag: {login} opened {issue_(issue)} {_repo(r)} {_title(issue)}')
    elif e.type == "IssueCommentEvent":
        issue = e.payload.issue
        show(term.white(f':speech_balloon: {login} commented on {issue_(issue)} {_repo(r)} {_title(issue)}'))
    elif e.type == "PushEvent":
        for c in e.payload.commits: commits_counter.update()
    elif e.type == "PullRequestEvent":
        pr_emoji,pr_color = ("white_heavy_check_mark",term.green) if e.payload.action == "closed" else ("sparkles",term.yellow)
        show(pr_color(f':{pr_emoji}: {login} {e.payload.action} a pull request {_repo(repo,20)} {_title(e.payload.pull_request)}'))
    elif e.type == "SecurityAdvisoryEvent": return print(term.blink("SECURITY ADVISORY"))

In [ ]:
#export    
def write_logs(events):
    Path("tmp.log").write_text(json.dumps(events, indent=2))
    shutil.move("tmp.log", logfile)

def tail_events():
    manager = enlighten.get_manager()
    commits = manager.counter(desc='Commits', unit='commits', color='green')
    while True:
        events = fetch_events()
        log = read_json_log(logfile)
        combined = sorted(log + events, key=lambda x: int(x["id"]))
        write_logs(combined)
        for x in combined: print_event(x, commits)
        time.sleep(0.2)

In [32]:
EVENT_EMOJI_MAPPING = defaultdict(lambda:'::', dict(
    CreateEvent=':new:',
    DeleteEvent=':x:',
    DeploymentEvent=':rocket:',
    ForkEvent=':fork_and_knife:',
    GistEvent=':notepad:',
    IssueCommentEvent=':speech_bubble:',
    LabelEvent=':label:',
    PushEvent=':fist:',
    ReleaseEvent=':rocket:',
    StarEvent=':star:',
    WatchEvent=':eyes:'
))

In [ ]:
def event_to_emoji(e):
    if EVENT_EMOJI_MAPPING[e["type"]] == '::': return e["type"]
    return emoji.emojize(EVENT_EMOJI_MAPPING[e["type"]], use_aliases=True)

In [ ]:
#export
def watch_users():
    users = {}
    users_events = {}
    while True:
        events = fetch_events()
        for x in events:
            login = x["actor"]["login"]
            if login in users: users[login] += 1
            else: users[login] = 1
            if login not in users_events: users_events[login] = {}
            if x['type'] not in users_events[login]: users_events[login][x['type']] = 1
            else: users_events[login][x['type']] += 1

        print (term.clear())
        print ("User".ljust(30), "Events".ljust(6), "PRs".ljust(5), "Issues".ljust(6), "Pushes".ljust(7))

        sorted_users = sorted(users.items(), key = lambda kv: (kv[1], kv[0]), reverse=True)
        for i in range(20):
            u = sorted_users[i]
            ue = users_events[u[0]]
            print(u[0].ljust(30), str(u[1]).ljust(6), 
                (str(ue['PullRequestEvent']) if 'PullRequestEvent' in ue else '').ljust(5), 
                (str(ue['IssuesEvent']) if 'IssuesEvent' in ue else '').ljust(6), 
                (str(ue['PushEvent']) if 'PushEvent' in ue else '').ljust(7))
        time.sleep(1)

In [ ]:
#export
def push_to_log(e):
    login,repo = e.actor.login,e.repo.name
    return "%s pushed %d commits to repo %s" % (login, len(e["payload"]["commits"]), repo)

In [ ]:
def issue_to_log(e):
    login,repo = e.actor.login,e.repo.name
    if e["type"] == "IssuesEvent":
        action = e["payload"]["action"]
        issue = e["payload"]["issue"]

        if action == 'closed':
            return emoji.emojize(':star:', use_aliases=True) + ' '  + login + ' closed issue #' + str(issue["number"]) + " on repo " + repo[:22] + " (\"" +  issue["title"][:50] + "...\")"
        elif action == 'opened':
            return emoji.emojize(':closed_mailbox_with_raised_flag:', use_aliases=True) + ' '  + login + ' opened issue #' + str(issue["number"]) + " on repo " + repo[:22] + " (\"" +  issue["title"][:50] + "...\")"

    elif e["type"] == "IssueCommentEvent":
        issue = e["payload"]["issue"]
        return emoji.emojize(':speech_balloon: ') + login + " commented on issue #" + str(issue["number"]) + " on repo " + repo[:22] + " (\"" +  issue["title"][:50] + "...\")"

In [ ]:
def pr_to_log(e):
    login,repo = e.actor.login,e.repo.name
    action = e["payload"]["action"]
    pr_emoji = ''
    pr_color = None
    if action == "closed":
        pr_emoji = emoji.emojize(":white_heavy_check_mark:")
    else:
        pr_emoji = emoji.emojize(":sparkles:")
    return pr_emoji + ' ' + login + " " + e["payload"]["action"] + " a pull request on repo " + repo[:20] + " (\"" +  e["payload"]["pull_request"]["title"][:50] + "...\")"

In [ ]:
#export
def release_to_log(e):
    login,repo = e.actor.login,e.repo.name
    tag = e["payload"]["release"]["tag_name"]
    return emoji.emojize(':rocket: ') + login + " released " + tag + " of " + repo

In [ ]:
#export
def str_clean(s): return s[:95]

In [ ]:
def signal_handler(sig, frame):
    if sig != signal.SIGINT:
        return
    term=Terminal()
    print(term.exit_fullscreen())
    print(term.clear())
    print(term.normal)
    sys.exit(0)

In [ ]:
#export
def quad_logs():
    term = Terminal()
    term.enter_fullscreen()

    ui = HSplit(
            VSplit(
                Log(title='Issues', border_color = 2, color=7),
                Log(title='Commits', border_color = 2, color=3)
            ),
            VSplit(
                Log(title='Pull Requests', border_color = 2, color=4),
                Log(title='Releases', border_color = 2, color=5)
            ),
        )

    issues = ui.items[0].items[0]
    commits = ui.items[0].items[1]
    prs = ui.items[1].items[0]
    releases = ui.items[1].items[1]

    issues.append(" ")
    commits.append(" ")
    prs.append(" ")
    releases.append(" ")

    while True:
        events = fetch_events()
        for x in events:
            t = x["type"]
            if t == 'PushEvent': commits.append(str_clean(push_to_log(x)))
            elif t == 'IssuesEvent' or t == 'IssueCommentEvent': issues.append(str_clean(issue_to_log(x)))
            elif t == 'PullRequestEvent': prs.append(str_clean(pr_to_log(x)))
            elif t == 'ReleaseEvent': releases.append(str_clean(release_to_log(x)))
        ui.display()
        time.sleep(0.1)

In [ ]:
#export
def simple():
    while True:
        events = fetch_events()
        for x in events:
            print("%s %s %s" % (x["actor"]["login"], x["type"], x["repo"]["name"]))

In [ ]:
if len(sys.argv) < 2: exit("Usage: ghtop <tail|quad|users|simple>")

signal.signal(signal.SIGINT, signal_handler)

if sys.argv[1] == 'tail': tail_events()
elif sys.argv[1] == 'quad': quad_logs()
elif sys.argv[1] == 'users': watch_users()
elif sys.argv[1] == 'simple': simple()